# <span style="color:#2834d4">역전파</span>

## 1. 연쇄법칙

연쇄 법칙: 합성합수의 미분은 합성함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.

## $\frac{\partial z}{\partial x} = \frac{\partial z}{\partial y} \frac{\partial y}{\partial x}$

## 2. add/mul/copy/max gate

<img src="./img/gates_gradient.png" width="50%" height="50%"/>

# <span style="color:#2834d4">단순한 계층 구현하기</span>

## 1. 곱셈 계층

### $z = xy$  
### $\frac{\partial z}{\partial x} = y$, $\frac{\partial z}{\partial y} = x$

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

<img src="./img/mullayer.png" width="70%" height="70%"/>

In [2]:
apple = 100
apple_num = 2
tax =1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [3]:
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


## 2. 덧셈 계층

### $z = x + y$
### $\frac{\partial z}{\partial x} = 1$, $\frac{\partial z}{\partial y} = 1$

In [4]:
class AddLayer:
    def __init_(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        
        return dx, dy

<img src="./img/addlayer.png" width="70%" height="70%"/>

In [5]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax =1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_prcie, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


# <span style="color:#2834d4">활성화 함수 계층 구현하기</span>

## 1. ReLU 계층

### $y = max(x, 0)$
### $\frac{\partial y}{\partial x} = \begin{cases} 1 & (x > 0) \\ 0 & (x \le 0) \end{cases} $ 

<img src="./img/relulayer.png" width="70%" height="70%"/>

In [6]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x<=0)  # x의 원소값 중 0 이하인 인덱스는 True, 그 외는 False
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0  # 입력 값이 0 이하면 역전파 때의 값은 0
        dx = dout
        
        return dx

## 2. sigmoid 계층

### $y = \frac{1}{1 + exp(-x)}$
- $y = \frac{1}{x}$ 
    - $\frac{\partial y}{\partial x} = -\frac{1}{x^2} = -y^2$
- $y = exp(x)$
    - $\frac{\partial y}{\partial x} = exp(x)$

<img src="./img/sigmoidlayer1.png" width="70%" height="70%" align='left'/><br>
<img src="./img/sigmoidlayer2.png" width="40%" height="40%" align='left'/><br>


### $\begin{matrix} \frac{\partial L}{\partial y} y^2 exp(-x) &=& \frac{\partial L}{\partial y} \frac{1}{(1+exp(-x))^2} exp(-x) \\ &=& \frac{\partial L}{\partial y} \frac{1}{1+exp(-x)} \frac{exp(-x)}{1+exp(-x)} \\ &=& \frac{\partial L}{\partial y} y (1-y) \end{matrix}$
<img src="./img/sigmoidlayer3.png" width="70%" height="70%" align='left'/>

In [7]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

## 3. Affine 계층 (배치용)

- Affine transformation : 신경망의 순전파 때 수행하는 행렬의 곱

### $\mathbf Y = \mathbf X \cdot \mathbf W + \mathbf B$
### $\frac{\partial L}{\partial \mathbf X} = \frac{\partial L}{\partial \mathbf Y} \cdot \mathbf W^T$, $\frac{\partial L}{\partial \mathbf W} = \mathbf X^T \cdot \frac{\partial L}{\partial \mathbf Y}$, $\frac{\partial L}{\partial \mathbf B} = \frac{\partial L}{\partial \mathbf Y}$ <br>

<img src="./img/affinelayer.png" width="70%" height="70%" align='left'/><br>

In [8]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

## 4. Softmax-with-Loss 계층

신경망에서 수행하는 작업: 학습, 추론
- 신경망 추론에서는 답을 하나만 내는 경우에는 가장 큰 값만 알면 되기 때문에 softmax 계층은 필요 없음
- 신경망을 학습할 때는 softmax 계층이 필요함
<br>
softmax function의 손실함수로 cross entropy error를 사용하면 역전파의 결과가 깔끔하게 떨어짐
<img src="./img/softmaxbackward.jpg" width="100%" height="100%" align='left'/><br>
<img src="./img/softmaxwithloss.png" width="70%" height="70%" align='left'/><br>

In [9]:
from common.layers import softmax

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None  # 손실
        self.y = None  # softmax의 출력
        self.t = None  # 정답 레이블(원-핫 벡터)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx

# <span style="color:#2834d4">오차역전파법 구현하기</span>

## 1. 오차역전파법을 적용한 신경망 구현하기

In [10]:
from common.functions import cross_entropy_error
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
        
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
    
    
    def loss(self, x, t):
        y = self.predict(x)
        
        return self.lastLayer.forward(y, t)
    
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1:
            t = np.argmax(t, axis=1)
            
        accuracy = np.sum(y==t) / float(x.shape[0])
        
        return accuracy
    
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)
        
        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

## 2. 오차역전파법을 사용한 학습 구현하기

In [11]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size/batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 오차역전파법으로 기울기 구하기
    grad = network.gradient(x_batch, t_batch)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.07788333333333333 0.0787
0.90005 0.9034
0.9235833333333333 0.9258
0.9364833333333333 0.9362
0.9464666666666667 0.9448
0.9541833333333334 0.9526
0.9571 0.954
0.9620833333333333 0.9574
0.9651166666666666 0.9603
0.9673 0.9628
0.9707 0.965
0.97225 0.9653
0.9748166666666667 0.9673
0.9752 0.9679
0.9774 0.9692
0.9786833333333333 0.9706
0.9799833333333333 0.9712


## 3. 오차역전파법으로 구한 기울기 검증하기

In [12]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_batch[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 차이의 절대값을 구한 후, 그 절대값들의 평균을 구함
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

W1:4.1015241101088167e-10
b1:2.4854103057928013e-09
W2:5.058266813590703e-09
b2:1.4045312727445358e-07


4.1015241101088167e-10 (0.00000000041) <br>
2.4854103057928013e-09 (0.00000000248) <br>
5.058266813590703e-09  (0.00000000505) <br>
1.4045312727445358e-07 (0.00000001404)

# <span style="color:#2834d4">정리</span>

- 계산 그래프를 이용하면 계산 과정을 시각적으로 표현 가능.
    - 계산 그래프의 노드는 국소적 계산으로 구성. 국소적 계산을 조합해 전체 계산을 구성.
- 계산 그래프의 역전파로 각 노드의 미분을 구할 수 있음.
- 신경망의 구성 요소를 계층적으로 구현해 기울기는 효율적으로 계산할 수 있음 (오차역전파법).
- 수치 미분과 오차역전파법의 결과를 비교하면 오차역전파법의 구현에 잘못이 없는지 확인할 수 있음 (기울기 확인).